In [3]:

import os
import shutil
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import load_model

# Video lesen
video_path = "Vehicle Detection Junktion 1.mp4"
cap = cv2.VideoCapture(video_path)

# Ordner erstellen, um Frames zu speichern
if not os.path.exists('frames'):
    os.makedirs('frames')

# Zähler initialisieren
count = 0

# Output-Ordner erstellen, falls er noch nicht existiert
output_folder = "output_folder"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Frames extrahieren
while True:
    # Frame lesen
    ret, frame = cap.read()

    # Überprüfen, ob Frame gelesen wurde
    if ret:
        # Wenn Zähler durch 25 teilbar, dann Frame speichern
        if count % 25 == 0:
            # Frame speichern
            frame_path = os.path.join('frames', f"frame{count}.jpg")
            cv2.imwrite(frame_path, frame)

            # Frame anzeigen
            cv2.imshow('Frame', frame)
            cv2.waitKey(1)

        # Zähler erhöhen
        count += 1
  
    # Wenn kein Frame gelesen wurde, dann Schleife beenden
    else:
        break
# Zähler für erkannte Fahrzeuge initialisieren
vehicle_count = 0

# Zähler für erkannte Fahrzeuge initialisieren
vehicle_count = 0

# Modell initialisieren oder aus Datei laden
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])
model = load_model('vehicle_detection_model.h5')

# Frames extrahieren
frames_folder = "frames"
for frame_name in os.listdir(frames_folder):
    frame_path = os.path.join(frames_folder, frame_name)

    # Bild lesen und skalieren
    img = cv2.imread(frame_path)
    img = cv2.resize(img, (64, 64))

    # Model laden
    model = load_model('vehicle_detection_model.h5')

    # Vorhersage auf dem Bild ausführen
    pred = model.predict(np.array([img]))
    pred_class = int(round(pred[0][0]))

    # Wenn ein Fahrzeug erkannt wurde, erhöhen Sie den Zähler
    if pred_class == 0:
        vehicle_count += 1

print("Anzahl der erkannten Fahrzeuge:", vehicle_count)

# Gesamtzahl der erkannten Fahrzeuge ausgeben
print("Gesamtzahl der erkannten Fahrzeuge:", vehicle_count)

# Output-Ordner erstellen, falls er noch nicht existiert
output_folder = "output_folder"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Schleife durchlaufen und Bilder der erkannten Fahrzeuge kopieren
for frame_name in os.listdir(frames):
    frame_path = os.path.join(frames_folder, frame_name)

    # Bild lesen und skalieren
    img = cv2.imread(frame_path)
    img = cv2.resize(img, (64, 64))

    # Vorhersage auf dem Bild ausführen
    pred = model.predict(np.array([img]))
    pred_class = int(round(pred[0][0]))

    # Wenn ein Fahrzeug erkannt wurde, speichern Sie das Bild im Output-Ordner
    if pred_class == 0:
        output_path = os.path.join(output_folder, frame_name)
        cv2.imwrite(output_path, img)

# Video-Objekt freigeben
cap.release()

# Alle Fenster schließen
cv2.destroyAllWindows()

# Eingabeaufforderung für Anzahl der Fahrzeuge
n = int(input("Wie viele Fahrzeuge möchten Sie klassifizieren? "))

# Initialisierung der Variablen für die Anzahl der LKWs, Busse und Pkws
num_lkw = 0
num_bus = 0
num_pkw = 0

# Schleife für die Eingabe und Klassifizierung jedes Fahrzeugs
for i in range(n):
    # Eingabeaufforderung für die Art des Fahrzeugs
    type = input("Geben Sie den Typ des Fahrzeugs (LKW, Bus oder Pkw) ein: ")

    # Klassifizierung des Fahrzeugs basierend auf dem eingegebenen Typ
    if type == "LKW":
        num_lkw += 1
    elif type == "Bus":
        num_bus += 1
    elif type == "Pkw":
        num_pkw += 1
    else:
        print("Ungültiger Fahrzeugtyp. Bitte geben Sie LKW, Bus oder Pkw ein.")
        continue

    # Hier kann man weitere Operationen durchführen, z.B. spezielle Aktionen für jeden Typ von Fahrzeugen

# Ausgabe der Anzahl der LKWs, Busse und Pkws
print("Anzahl der LKWs:", num_lkw)
print("Anzahl der Busse:", num_bus)
print("Anzahl der Pkws:", num_pkw)


# Daten und Labels laden
frames_folder = "frames"
X = []
y = []

for frame_name in os.listdir(frames_folder):
    frame_path = os.path.join(frames_folder, frame_name)

    # Bild lesen und skalieren
    img = cv2.imread(frame_path)
    img = cv2.resize(img, (64, 64))

    # Label aus Dateinamen extrahieren
    label = 0 if "car" in frame_name else 1

    X.append(img)
    y.append(label)

X = np.array(X)
y = np.array(y)

# Trennen von Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Definieren Sie das CNN-Modell
model = Sequential()

# Fügen Sie den ersten Convolutional Layer hinzu
model.add(Conv2D(filters=32, kernel_size=(3, 3),
          activation='relu', input_shape=(64, 64, 3)))

# Fügen Sie den ersten Max-Pooling Layer hinzu
model.add(MaxPooling2D(pool_size=(2, 2)))

# Fügen Sie den zweiten Convolutional Layer hinzu
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))

# Fügen Sie den zweiten Max-Pooling Layer hinzu
model.add(MaxPooling2D(pool_size=(2, 2)))

# Fügen Sie den Flatten Layer hinzu
model.add(Flatten())

# Fügen Sie den ersten Dense Layer hinzu
model.add(Dense(units=128, activation='relu'))

# Fügen Sie den Ausgabe-Layer hinzu
model.add(Dense(units=1, activation='sigmoid'))

# Kompilieren Sie das Modell
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

# Trainieren Sie das Modell
model.fit(X_train, y_train, epochs=10)

# Modell speichern
model.save('vehicle_detection_model.h5')


# Laden des trainierten Modells
model = load_model('vehicle_detection_model.h5')

# Vorhersage auf Testdatensatz ausführen
y_pred = model.predict(X_test)

# Ausgabe der Vorhersage
for i in range(len(y_pred)):
    print(f"Actual: {y_test[i]}, Predicted: {int(round(y_pred[i][0]))}")


1/1 [==============================] - 0s 46ms/step
Anzahl der erkannten Fahrzeuge: 0
Gesamtzahl der erkannten Fahrzeuge: 0


NameError: name 'frames' is not defined